- **API를 활용한 긴 텍스트 번역 순서**
  - 라이브러리 임포트
  - 텍스트 파일 읽기 및 분리
  - 데이터베이스 설정
  - 데이터 삽입 및 로드

  - **데이터 준비**
    - 텍스트 파일 읽기 및 분리
    - 데이터베이스 설정
    - 데이터 삽입 및 로드

- **OpenAI API 설정**
  - 번역 함수 정의 및 테스트
  - 번역되지 않은 데이터 로드

  - **번역 과정**
    - OpenAI API 설정
    - 번역 함수 정의 및 테스트
    - 번역되지 않은 데이터 로드

- **번역 및 완료**
  - 데이터 번역 및 업데이트
  - 데이터베이스 커밋 및 종료

### Prompt
* 텍스트 파일을 열어 개행 기준으로 데이터를 나누고 sqlite 파일로 저장할것
* 이 때 컬럼을  text, length, gpt4omini_trans, gpt4omini_trans 로 만들고 news_text 에는 나눈 텍스트 데이터, length 에는 해당 텍스트 길이가 들어가도록 하고 나머지 컬럼은 비워둘 것
* 주석은 한국어로 작성하고 데이터프레임을 불러와서 df 변수에 담고 확인하는 코드를 함께 작성할 것

In [ ]:
import os
import sqlite3

def get_first_txt_file(directory='.'):
    txt_files = [f for f in os.listdir(directory) if f.endswith('yp6WuHFhYCo_subtitle.txt')]
    txt_files.sort()  # 파일명 알파벳 순으로 정렬
    return txt_files[0] if txt_files else None

def create_chunks(text, chunk_size=1000):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def create_db_and_table():
    conn = sqlite3.connect('text_chunks.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS chunks
                      (id INTEGER PRIMARY KEY,
                       original_text TEXT,
                       length INTEGER,
                       translation TEXT)''')
    conn.commit()
    return conn, cursor

def insert_chunks(cursor, chunks):
    for chunk in chunks:
        cursor.execute('''INSERT INTO chunks (original_text, length, translation)
                          VALUES (?, ?, ?)''', (chunk, len(chunk), None))

def main():
    # 첫 번째 .txt 파일 찾기
    first_txt_file = get_first_txt_file()
    if not first_txt_file:
        print("No .txt files found in the current directory.")
        return

    # 텍스트 파일 읽기
    with open(first_txt_file, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # 줄바꿈 문자로 분리
    lines = text.split('\n')
    
    # 1000자 단위로 청크 생성
    chunks = []
    current_chunk = ""
    for line in lines:
        if len(current_chunk) + len(line) + 1 > 1000:
            chunks.extend(create_chunks(current_chunk))
            current_chunk = line + '\n'
        else:
            current_chunk += line + '\n'
    
    if current_chunk:
        chunks.extend(create_chunks(current_chunk))
    
    # DB 생성 및 연결
    conn, cursor = create_db_and_table()
    
    # 청크 삽입
    insert_chunks(cursor, chunks)
    
    # 변경사항 저장 및 연결 종료
    conn.commit()
    conn.close()

    print(f"파일 '{first_txt_file}'에서 총 {len(chunks)}개의 청크가 DB에 저장되었습니다.")

if __name__ == "__main__":
    main()

In [ ]:
# 저장된 데이터를 pandas DataFrame으로 읽어오기
import pandas as pd

# SQLite 데이터베이스에 다시 연결
conn = sqlite3.connect('text_chunks.db')

# SQL 쿼리를 사용하여 'chunks' 테이블의 모든 데이터를 DataFrame으로 읽어오기
df = pd.read_sql_query("SELECT * FROM chunks", conn)

# 연결 종료
conn.close()

# DataFrame의 처음 몇 행 출력하여 확인
df.head()

In [ ]:
# DataFrame의 정보 출력
df.info()